1단계: 📁 데이터 준비 및 폴더 구조 확인<Br>
PyTorch와 torchvision 라이브러리를 사용하여 이미지 분류를 수행하려면, 데이터 폴더 구조가 표준 형식(ImageFolder)을 따라야 합니다.

✅ 폴더 구조 (예시)<Br>
화살표 분류 프로젝트의 경우, train, val, test 세트 내부에 클래스 이름(예: 0 또는 1)으로 된 하위 폴더가 있어야 합니다.

In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# 1. 하이퍼파라미터 및 경로 설정
# 현재 사용하시는 경로에 맞춰 'root_dir'을 설정해 주세요.
ROOT_DIR = 'C:/Dev/KAIROS_Project_1/ResNet50/data' 
BATCH_SIZE = 32 # VRAM 환경에 따라 조정 (YOLO보다 더 많은 메모리를 요구할 수 있음)
NUM_WORKERS = 4 # 데이터 로드 속도를 위한 CPU 코어 수 (환경에 따라 0으로 설정 가능)
IMG_SIZE = 224 # ResNet-50 표준 입력 크기 (224x224)

# ResNet의 ImageNet 사전 학습 가중치에 맞춰 정규화 표준 사용
NORM_MEAN = [0.485, 0.456, 0.406]
NORM_STD = [0.229, 0.224, 0.225]

# 2. 전처리 파이프라인 정의
transform = transforms.Compose([
    # 이미지 크기를 256x256으로 조절
    transforms.Resize(256),
    # 224x224 크기로 중앙을 잘라냄 (Crop)
    transforms.CenterCrop(IMG_SIZE), 
    # PIL Image를 PyTorch Tensor로 변환
    transforms.ToTensor(),
    # 정규화
    transforms.Normalize(NORM_MEAN, NORM_STD)
])

# 3. 데이터셋 생성 및 로드
def create_dataloaders(root_dir, batch_size, num_workers, transform):
    """
    train, val, test 데이터 로더를 생성하는 함수
    """
    # ImageFolder: 폴더 구조를 기반으로 데이터와 레이블을 자동 로드
    train_dataset = ImageFolder(root=f'{root_dir}/train', transform=transform)
    val_dataset = ImageFolder(root=f'{root_dir}/val', transform=transform)
    test_dataset = ImageFolder(root=f'{root_dir}/test', transform=transform)

    # 데이터 로더 생성
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    
    # 클래스 이름 확인
    print(f"✅ 클래스: {train_dataset.classes}")
    print(f"✅ 학습 데이터셋 크기: {len(train_dataset)}")
    
    return train_loader, val_loader, test_loader

# 데이터 로더 실행
train_loader, val_loader, test_loader = create_dataloaders(ROOT_DIR, BATCH_SIZE, NUM_WORKERS, transform)

[확인 사항] 현재 YOLO 프로젝트에서 사용하던 train/images, val/images가 아닌, 클래스별로 이미지를 분류한 위와 같은 구조로 데이터를 이동하거나 복사했는지 확인해 주세요.

2단계: ⚙️ 데이터 로더(DataLoader) 구현<br>
데이터 로더는 이미지 파일을 읽고 전처리하며, 모델 학습에 필요한 배치(Batch) 단위로 데이터를 공급하는 역할을 합니다.

In [ ]:
import torch.nn as nn
from torchvision import models

if torch.cuda.is_available():
    # GPU가 사용 가능하면 인덱스 0번 GPU를 사용
    device = torch.device("cuda:0")
    print(f"✅ 사용 디바이스: {torch.cuda.get_device_name(0)}")
else:
    # GPU가 없으면 CPU 사용
    device = torch.device("cpu")
    print(f"✅ 사용 디바이스: {device}")

# 2. 사전 학습된 ResNet-50 모델 로드
# weights='ResNet50_Weights.IMAGENET1K_V1'는 최신 PyTorch에서 ImageNet 가중치를 로드하는 표준 방식입니다.
model = models.resnet50(weights='ResNet50_Weights.IMAGENET1K_V1')

# 3. 전이 학습을 위한 모델 수정
# ResNet-50의 마지막 Fully Connected 레이어(fc)는 기본 1000개의 클래스(ImageNet)로 되어 있음
num_ftrs = model.fc.in_features # fc 레이어의 입력 피처 수 확인

# 프로젝트의 클래스 수(2개: '0', '1')에 맞게 새로운 fc 레이어로 교체
model.fc = nn.Linear(num_ftrs, 2) 

# 4. 모델을 지정된 디바이스로 이동
model = model.to(device)

print(f"✅ 모델 로드 및 수정 완료. 최종 출력 피처 수: {model.fc.out_features}")

3단계: 🧠 ResNet-50 모델 로드 및 수정<br>
ResNet-50의 사전 학습된 가중치를 가져와, 화살표 두 클래스(0, 1)에 맞게 출력 레이어(fc 레이어)를 수정합니다.

In [ ]:
import torch.nn as nn
from torchvision import models

# 1. 디바이스 설정 (YOLO와 동일하게 device=1을 가정)
device = torch.device("cuda:0" if torch.cuda.is_available() and torch.cuda.device_count() >= 1 else "cpu")
print(f"✅ 사용 디바이스: {device}")

# 2. 사전 학습된 ResNet-50 모델 로드
# weights='ResNet50_Weights.IMAGENET1K_V1'는 최신 PyTorch에서 ImageNet 가중치를 로드하는 표준 방식입니다.
model = models.resnet50(weights='ResNet50_Weights.IMAGENET1K_V1')

# 3. 전이 학습을 위한 모델 수정
# ResNet-50의 마지막 Fully Connected 레이어(fc)는 기본 1000개의 클래스(ImageNet)로 되어 있음
num_ftrs = model.fc.in_features # fc 레이어의 입력 피처 수 확인

# 프로젝트의 클래스 수(2개: '0', '1')에 맞게 새로운 fc 레이어로 교체
model.fc = nn.Linear(num_ftrs, 2) 

# 4. 모델을 지정된 디바이스로 이동
model = model.to(device)

print(f"✅ 모델 로드 및 수정 완료. 최종 출력 피처 수: {model.fc.out_features}")

4단계: 📈 손실 함수(Loss) 및 최적화 도구(Optimizer) 정의<br>
분류 문제에 적합한 도구를 설정합니다.

In [ ]:
import torch.optim as optim

# 손실 함수: 분류에 주로 사용되는 교차 엔트로피(Cross Entropy)
criterion = nn.CrossEntropyLoss()

# 최적화 도구: AdamW 또는 SGD가 일반적으로 좋은 성능을 보임
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# 학습률 스케줄러 (선택 사항): 학습이 진행됨에 따라 학습률을 점진적으로 낮춰 성능 개선
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

5단계: 🏃 학습 루프(Training Loop) 구현<br>
실제 모델 학습이 진행되는 반복 루프입니다.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import models
import time
import os
import numpy as np
from sklearn.metrics import confusion_matrix, f1_score # F1-Score 계산을 위해 추가
from typing import Tuple, List, Dict 

# ==============================================================================
# 1. 설정 및 하이퍼파라미터
# ==============================================================================
# ⚠️ 중요: 'ROOT_DIR'을 'data' 폴더가 있는 실제 경로로 수정하세요.
ROOT_DIR = 'C:/Dev/KAIROS_Project_1/ResNet50/data' 
BATCH_SIZE = 32     
NUM_CLASSES = 2     
NUM_EPOCHS = 500    # 요청하신 500 에포크로 설정
IMG_SIZE = 224      
LR = 0.001          

# ImageNet 표준 정규화 값
NORM_MEAN = [0.485, 0.456, 0.406]
NORM_STD = [0.229, 0.224, 0.225]

# ==============================================================================
# 2. 디바이스 설정 (GPU 사용 최적화)
# ==============================================================================
if torch.cuda.is_available():
    device = torch.device("cuda:0") 
    print(f"✅ 사용 디바이스: {torch.cuda.get_device_name(0)} (cuda:0)")
else:
    device = torch.device("cpu")
    print("✅ 사용 디바이스: cpu")


# ==============================================================================
# 3. 전처리 및 데이터 로더 생성
# ==============================================================================
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(IMG_SIZE),        
    transforms.RandomHorizontalFlip(),      
    transforms.ToTensor(),
    transforms.Normalize(NORM_MEAN, NORM_STD)
])

val_test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(IMG_SIZE), 
    transforms.ToTensor(),
    transforms.Normalize(NORM_MEAN, NORM_STD)
])

train_dataset = ImageFolder(root=os.path.join(ROOT_DIR, 'train'), transform=train_transform)
val_dataset = ImageFolder(root=os.path.join(ROOT_DIR, 'val'), transform=val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

print(f"✅ 클래스: {train_dataset.classes}")
print(f"✅ 학습/검증 데이터셋 크기: {len(train_dataset)} / {len(val_dataset)}")


# ==============================================================================
# 4. ResNet-50 모델 로드, 수정, 학습 환경 정의
# ==============================================================================
model = models.resnet50(weights='ResNet50_Weights.IMAGENET1K_V1')

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, NUM_CLASSES) 

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=LR)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) 


# ==============================================================================
# 5. 학습 루프 구현 및 F1-Score/Accuracy 기반 모델 저장 (수정됨)
# ==============================================================================
# ... (상위 import 및 설정 부분은 이전 코드와 동일) ...

timestamp = time.strftime('%Y%m%d_%H%M%S')

def train_model(model, criterion, optimizer, scheduler, num_epochs):
    start_time = time.time()
    best_f1_score = -1.0  # 🌟 최고 F1-Score 추적
    best_acc = -1.0       # 🌟 최고 Accuracy 추적
    
    save_dir = './resnet_saved_models'
    os.makedirs(save_dir, exist_ok=True)
    
    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print('-' * 20)

        for phase in ['train', 'val']:
            # ... (모델 모드 설정 및 데이터 로더 지정) ...
            if phase == 'train':
                model.train()  
                dataloader = train_loader
            else:
                model.eval()   
                dataloader = val_loader

            running_loss = 0.0
            running_corrects = 0
            
            all_preds = []   
            all_labels = []  

            for inputs, labels in dataloader:
                # ... (순전파, 역전파 및 통계 계산 부분은 이전 코드와 동일) ...
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad() 

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1) 
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())


            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            # 🌟🌟 저장 로직 (검증 단계에서만 실행)
            if phase == 'val':
                
                current_f1_score = f1_score(all_labels, all_preds, zero_division=0) 
                current_acc = epoch_acc.item() # tensor에서 float으로 변환
                
                # 로그 출력용 CM 계산
                try:
                    cm = confusion_matrix(all_labels, all_preds)
                    # cm.ravel() = [TN, FP, FN, TP]
                    tn, fp, fn, tp = cm.ravel()
                    cm_log = f" (TN:{tn}, FP:{fp}, FN:{fn}, TP:{tp}, F1:{current_f1_score:.4f})"
                except ValueError:
                    cm_log = f" (F1:{current_f1_score:.4f})"
                
                print(f"  [Metric Score] F1:{current_f1_score:.4f}, Acc:{current_acc:.4f}{cm_log}")
                
                
                # 📌 저장 조건 1: F1과 Acc가 모두 1.0일 때 (완벽한 성능)
                if current_f1_score >= 1.0 and current_acc >= 1.0:
                    best_f1_score = current_f1_score # 최고 기록 업데이트
                    best_acc = current_acc
                    
                    # 파일 이름을 'PERFECT_F1_Acc_에포크번호_타임스탬프.pth' 형식으로 지정
                    model_save_path = os.path.join(save_dir, f'resnet50_PERFECT_e{epoch+1}_f1_1.0000_{timestamp}.pth') 
                    
                    torch.save(model.state_dict(), model_save_path)
                    
                    print(f"🎉🎉 퍼펙트 스코어 달성! 모델 가중치 저장 완료: {model_save_path}")

                
                # 📌 저장 조건 2: 최고 기록 경신 (F1 우선, Acc는 F1이 같을 때만 비교)
                elif current_f1_score > best_f1_score or (current_f1_score == best_f1_score and current_acc > best_acc):
                    
                    best_f1_score = current_f1_score
                    best_acc = current_acc
                    
                    # 파일 이름을 'BEST_F1_Acc_타임스탬프.pth' 형식으로 지정
                    model_save_path = os.path.join(save_dir, f'resnet50_BEST_e{epoch+1}_f1_{best_f1_score:.4f}_acc_{best_acc:.4f}_{timestamp}.pth') 
                    
                    torch.save(model.state_dict(), model_save_path)
                    
                    print(f"🌟 최고 성능 경신 (F1:{best_f1_score:.4f}, Acc:{best_acc:.4f}), 모델 가중치 저장 완료: {model_save_path}")
                
                else:
                    print(f"❗ 최고 성능 경신 실패 (Best F1:{best_f1_score:.4f}, Best Acc:{best_acc:.4f})")


    time_elapsed = time.time() - start_time
    print(f'총 학습 시간: {time_elapsed // 60:.0f}분 {time_elapsed % 60:.0f}초')
    print(f'최종 최고 F1-Score: {best_f1_score:.4f}, 최종 최고 Accuracy: {best_acc:.4f}')
    return model

# 🚀 학습 시작
final_model = train_model(model, criterion, optimizer, scheduler, 500)

NameError: name 'time' is not defined